In [1]:
import networkx as nx
import numpy as np
import os

In [2]:
path_old_batch = './OASIS_full_batch/modified_graphs/'
path_new_batch = './Oasis_original_new/'
path_to_save = './Oasis_original_new_with_dummy/'

In [3]:
old_graphs = [nx.read_gpickle(path_old_batch + graph) for graph in os.listdir(path_old_batch)]
new_graphs = [nx.read_gpickle(path_new_batch + graph) for graph in os.listdir(path_new_batch)]

new_graph_sizes = [nx.number_of_nodes(g) for g in new_graphs]

In [4]:
# old_graphs[0].nodes.data()

In [5]:
def get_geodesic_distance_sphere(coord_a, coord_b, radius):
    ''' 
    Return the geodesic distance of two 3D vectors on a sphere
    '''
    return radius * np.arccos(np.clip(np.dot(coord_a, coord_b) / np.power(radius,2),-1,1))

In [6]:
def add_geodesic_distance_on_edges(graph):
    """
    Compute the geodesic distance represented by each edge
    and add it as attribute in the graph
    """

    # initialise the dict for atttributes on edges
    edges_attributes = {}

    # Fill the dictionnary with the geodesic_distance
    for edge in graph.edges:
        geodesic_distance = get_geodesic_distance_sphere(graph.nodes[edge[0]]["sphere_3dcoords"],
                                                         graph.nodes[edge[1]]["sphere_3dcoords"],
                                                         radius=100)

        edges_attributes[edge] = {"geodesic_distance": geodesic_distance}

    nx.set_edge_attributes(graph, edges_attributes)

In [7]:
def add_id_on_edges(graph):
    """
    Add an Id information on edge (integer)
    """

    # initialise the dict for atttributes on edges
    edges_attributes = {}

    # Fill the dictionnary with the geodesic_distance
    for i, edge in enumerate(graph.edges):
        edges_attributes[edge] = {"id": i}

    nx.set_edge_attributes(graph, edges_attributes)
    

In [8]:
def transform_3d_coordinates_into_ndarray(graph):
    """
    Transform the node attribute sphere_3dcoord from a list to a ndarray
    """
    # initialise the dict for atttributes on edges
    nodes_attributes = {}

    # Fill the dictionnary with the nd_array attribute
    for node in graph.nodes:
        nodes_attributes[node] = {"sphere_3dcoords": np.array(graph.nodes[node]["sphere_3dcoords"])}

    nx.set_node_attributes(graph, nodes_attributes)

In [9]:
def dummy_for_visu(g,max_size):
    len_g = len(g.nodes)
    num_nodes_add = max_size - len_g
    
    
    dummy_dict = {}
    for i in range(nx.number_of_nodes(g)):
        dummy_dict[i] = {'is_dummy':False}
        
    nx.set_node_attributes(g,dummy_dict)    
        
    if num_nodes_add > 0.:
        for i in range(num_nodes_add):
            g.add_node(len_g+i,is_dummy = True)

In [10]:
all_graphs_name = os.listdir(path_new_batch)

In [11]:
# Process graphs

# Transform the 3d attributes into ndarray
for graph in new_graphs:
    transform_3d_coordinates_into_ndarray(graph)

# Calculate the geodesic distance for each node and add the id information
for graph in new_graphs:
    add_geodesic_distance_on_edges(graph)
    add_id_on_edges(graph)


for i,graph in enumerate(new_graphs):
    
    graph_num = int(all_graphs_name[i].split('_')[1].split('.')[0])
    
    max_size = max(new_graph_sizes)
    dummy_for_visu(graph,max_size)
    
                   
    nx.write_gpickle(graph, os.path.join(path_to_save, "modified_graphs","graph_{:05d}".format(graph_num) + ".gpickle"))

In [26]:
old_graphs[33].nodes.data()[0]

{'vertex_index': 3286,
 'sphere_3dcoords': array([  5.03527355, -95.44298553, -29.41569519]),
 'sphere_coords': [-1.5180883407592773, 1.869369626045227],
 'basin_label': 235.0,
 'basin_area': 1035.7528,
 'depth': -0.5467673,
 'basin_thickness': 2.415462017059326,
 'ico100_7_vertex_index': 7573,
 'is_dummy': False,
 'label_dbscan': -1}

In [25]:
new_graphs[34].nodes.data()[0]

{'vertex_index': 1916,
 'sphere_3dcoords': array([  4.7382164 , -92.68218231, -37.25      ]),
 'sphere_coords': [-1.5197175741195679, 1.9524977207183838],
 'basin_label': 459.0,
 'basin_area': 114.99903,
 'depth': 0.65721834,
 'basin_thickness': 2.067657232284546,
 'ico100_7_vertex_index': 30281,
 'label_neuroimage': 28,
 'label_media': 90,
 'sphere_3dcoords_noreg': array([ 16.458645, -86.622185, -47.17743 ], dtype=float32),
 'ico100_7_vertex_index_noreg': 1896,
 'is_dummy': False}